### 测试stdout输出

使用 unittest.mock 模块的 patch() 方法可以很方便的在测试运行的上下文中替换对象

In [2]:
# mymodule.py

def urlprint(protocol, host, domain):
    url = '{}://{}.{}'.format(protocol, host, domain)
    print(url)

In [3]:
from io import StringIO
from unittest import TestCase
from unittest.mock import patch
import mymodule

class TestURLPrint(TestCase):
    def test_url_gets_to_stdout(self):
        protocol = 'http'
        host = 'www'
        domain = 'example.com'
        expected_url = '{}://{}.{}\n'.format(protocol, host, domain)

        with patch('sys.stdout', new=StringIO()) as fake_out:
            mymodule.urlprint(protocol, host, domain)
            self.assertEqual(fake_out.getvalue(), expected_url)

### 在单元测试中给对象打补丁

In [ ]:
from unittest.mock import patch
import example

# 装饰器
@patch('example.func')
def test1(x, mock_func):
    example.func(x)       # Uses patched example.func
    mock_func.assert_called_with(x)
    
# 上下文
with patch('example.func') as mock_func:
    example.func(x)      # Uses patched example.func
    mock_func.assert_called_with(x)
    
# 手动的使用
p = patch('example.func')
mock_func = p.start()
example.func(x)
mock_func.assert_called_with(x)
p.stop()

### 在单元测试中测试异常情况

In [ ]:
import unittest

# A simple function to illustrate
def parse_int(s):
    return int(s)

class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        self.assertRaises(ValueError, parse_int, 'N/A')

In [ ]:
import errno

class TestIO(unittest.TestCase):
    def test_file_not_found(self):
        try:
            f = open('/file/not/found')
        except IOError as e:
            self.assertEqual(e.errno, errno.ENOENT)

        else:
            self.fail('IOError not raised')

### 将测试输出用日志记录到文件中

In [ ]:
import unittest
import sys

class MyTest(unittest.TestCase):
    pass

def main(out=sys.stderr, verbosity=2):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromModule(sys.modules[__name__])
    unittest.TextTestRunner(out, verbosity=verbosity).run(suite)

if __name__ == '__main__':
    with open('testing.out', 'w') as f:
        main(f)

### 忽略或期望测试失败

In [ ]:
import unittest
import os
import platform

class Tests(unittest.TestCase):
    def test_0(self):
        self.assertTrue(True)

    @unittest.skip('skipped test')
    def test_1(self):
        self.fail('should have failed!')

    @unittest.skipIf(os.name=='posix', 'Not supported on Unix')
    def test_2(self):
        import winreg

    @unittest.skipUnless(platform.system() == 'Darwin', 'Mac specific test')
    def test_3(self):
        self.assertTrue(True)

    @unittest.expectedFailure
    def test_4(self):
        self.assertEqual(2+2, 5)

if __name__ == '__main__':
    unittest.main()

### 处理多个异常

In [ ]:
try:
    client_obj.get_url(url)
except (URLError, ValueError, SocketTimeout):
    client_obj.remove_url(url)

In [ ]:
try:
    client_obj.get_url(url)
except (URLError, ValueError):
    client_obj.remove_url(url)
except SocketTimeout:
    client_obj.handle_url_timeout(url)

### 捕获所有异常

这个将会捕获除了 SystemExit 、 KeyboardInterrupt 和 GeneratorExit 之外的所有异常。 如果你还想捕获这三个异常，将 Exception 改成 BaseException 即可。

In [ ]:
try:
   ...
except Exception as e:
   ...
   log('Reason:', e)       # Important!

### 创建自定义异常

In [ ]:
class NetworkError(Exception):
    pass

class HostnameError(NetworkError):
    pass

class CustomError(Exception):
    def __init__(self, message, status):
        super().__init__(message, status)
        self.message = message
        self.status = status

### 捕获异常后抛出另外的异常

In [ ]:
def example():
    try:
            int('N/A')
    except ValueError as e:
            raise RuntimeError('A parsing error occurred') from e

In [ ]:
try:
    example()
except RuntimeError as e:
    print("It didn't work:", e)

    if e.__cause__:
        print('Cause:', e.__cause__)

### 重新抛出被捕获的异常

In [ ]:
def example():
    try:
            int('N/A')
    except ValueError:
            print("Didn't work")
            raise

### 输出警告信息

In [ ]:
import warnings

def func(x, y, logfile=None, debug=False):
    if logfile is not None:
         warnings.warn('logfile argument deprecated', DeprecationWarning)

### 调试基本的程序崩溃错误

如果你的程序因为某个异常而崩溃，运行 python3 -i someprogram.py 可执行简单的调试

### 给你的程序做性能测试

In [ ]:
bash % time python3 someprogram.py

In [ ]:
bash % python3 -m cProfile someprogram.py

In [ ]:
# timethis.py

import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        r = func(*args, **kwargs)
        end = time.perf_counter()
        print('{}.{} : {}'.format(func.__module__, func.__name__, end - start))
        return r
    return wrapper

@timethis
def countdown(n):
    while n > 0:
            n -= 1

In [ ]:
from contextlib import contextmanager

@contextmanager
def timeblock(label):
    start = time.perf_counter()
    try:
        yield
    finally:
        end = time.perf_counter()
        print('{} : {}'.format(label, end - start))
        
with timeblock('counting'):
    n = 10000000
    while n > 0:
            n -= 1

In [ ]:
timeit('math.sqrt(2)', 'import math', number=10000000)
# 1.434852126003534

In [ ]:
# 执行时间 time.process_time()
from functools import wraps
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.process_time()
        r = func(*args, **kwargs)
        end = time.process_time()
        print('{}.{} : {}'.format(func.__module__, func.__name__, end - start))
        return r
    return wrapper

### 加速程序运行

- 很少有人知道，像这样定义在全局范围的代码运行起来要比定义在函数中运行慢的多。 这种速度差异是由于局部变量和全局变量的实现方式（使用局部变量要更快些），使用函数带来15-30%的性能提升是很常见的

- 每一次使用点(.)操作符来访问属性的时候会带来额外的开销。 它会触发特定的方法，比如 __getattribute__() 和 __getattr__() ，这些方法会进行字典操作操作

- 任何时候当你使用额外的处理层（比如装饰器、属性访问、描述器）去包装你的代码时，都会让程序运行变慢

- 内置的数据类型比如字符串、元组、列表、集合和字典都是使用C来实现的，运行起来非常快。